# **Notebook Purpose**
*In this notebook we will build matchmaking function to match tourist with guides using persona classification and pca model that we build*

# **Import Library**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.metrics.pairwise import euclidean_distances

# **Tidying Up Datasets**

## Load Datasets

In [2]:
# Load Tour Guide Datasets
guide_df = pd.read_csv('data/tour-guide-data.csv')
guide_df

,Name,Gender,Domicile,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Nabhan Nabilah,1,0,1,0,0,0,0,0,0,...,5,2,5,5,5,2,2,5,5,2
1,Salsabilla,1,0,1,0,0,0,0,0,1,...,4,3,3,4,4,2,2,2,4,2
2,Maulani,1,0,1,0,0,0,0,1,0,...,4,4,3,4,4,3,3,2,3,3
3,Abiyyu Farhan,0,0,1,0,0,0,0,0,1,...,4,5,3,5,4,4,4,4,4,4
4,Rahmaliyah Kadir,1,1,1,0,0,0,0,0,0,...,4,3,3,4,3,3,3,3,4,3
5,Faizah,1,1,1,0,0,0,0,0,0,...,4,2,5,5,5,3,4,5,5,4
6,Andani,1,1,0,1,0,0,0,0,1,...,4,4,4,3,3,4,3,3,3,3
7,Muthiah Hanun,1,1,0,1,0,0,0,0,0,...,5,5,5,5,5,5,4,3,5,4
8,Ihlasul Mufti Faqih,0,1,1,0,0,0,0,0,1,...,5,4,5,4,3,2,1,3,4,2
9,Dessylva Maharany Santosa,1,0,1,0,0,0,0,1,0,...,3,3,3,4,3,2,3,2,4,3


In [3]:
# Add roles column as a guide
guide_df['Roles'] = 'guide'
guide_df

,Name,Gender,Domicile,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Nabhan Nabilah,1,0,1,0,0,0,0,0,0,...,2,5,5,5,2,2,5,5,2,guide
1,Salsabilla,1,0,1,0,0,0,0,0,1,...,3,3,4,4,2,2,2,4,2,guide
2,Maulani,1,0,1,0,0,0,0,1,0,...,4,3,4,4,3,3,2,3,3,guide
3,Abiyyu Farhan,0,0,1,0,0,0,0,0,1,...,5,3,5,4,4,4,4,4,4,guide
4,Rahmaliyah Kadir,1,1,1,0,0,0,0,0,0,...,3,3,4,3,3,3,3,4,3,guide
5,Faizah,1,1,1,0,0,0,0,0,0,...,2,5,5,5,3,4,5,5,4,guide
6,Andani,1,1,0,1,0,0,0,0,1,...,4,4,3,3,4,3,3,3,3,guide
7,Muthiah Hanun,1,1,0,1,0,0,0,0,0,...,5,5,5,5,5,4,3,5,4,guide
8,Ihlasul Mufti Faqih,0,1,1,0,0,0,0,0,1,...,4,5,4,3,2,1,3,4,2,guide
9,Dessylva Maharany Santosa,1,0,1,0,0,0,0,1,0,...,3,3,4,3,2,3,2,4,3,guide


In [4]:
# Load Tourist Datasets
tourist_df = pd.read_csv('data/tourist-data.csv')
tourist_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,5,4,4,5,5,4,4,2,4,4
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,5,4,3,3,4,3
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,2,3,2,3,4,3,2,3,4,3
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,3,5,3,4,5,4,4,1,4,4
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Féi,1,1,1,0,0,0,0,0,0,...,4,2,4,4,4,3,4,2,4,4
69,Annisa,1,1,1,0,0,0,0,0,0,...,4,2,3,3,3,3,3,2,4,3
70,Usamah,0,1,1,0,0,0,0,0,1,...,5,5,4,5,5,4,4,5,5,5
71,Helena,1,0,0,0,0,0,0,1,0,...,4,3,4,4,4,3,4,2,5,4


In [5]:
# Add roles columns as a tourist
tourist_df['Roles'] = 'tourist'
tourist_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,4,4,5,5,4,4,2,4,4,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,4,3,3,4,3,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,3,2,3,4,3,2,3,4,3,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,5,3,4,5,4,4,1,4,4,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,3,3,4,4,2,3,3,2,3,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Féi,1,1,1,0,0,0,0,0,0,...,2,4,4,4,3,4,2,4,4,tourist
69,Annisa,1,1,1,0,0,0,0,0,0,...,2,3,3,3,3,3,2,4,3,tourist
70,Usamah,0,1,1,0,0,0,0,0,1,...,5,4,5,5,4,4,5,5,5,tourist
71,Helena,1,0,0,0,0,0,0,1,0,...,3,4,4,4,3,4,2,5,4,tourist


## Combine Datasets Into Users Datasets

In [6]:
# Check if column names are the same
columns_equal = guide_df.columns.equals(tourist_df.columns)

if columns_equal:
    print("The DataFrames have the same column names.")
else:
    print("The DataFrames do not have the same column names.")

The DataFrames do not have the same column names.


In [7]:
# Get column names from guide_df and tourist_df
guide_columns = set(guide_df.columns)
tourist_columns = set(tourist_df.columns)

# Find the different columns
diff_col_in_guide = guide_columns - tourist_columns
diff_col_in_tourist = tourist_columns - guide_columns

# Print the different columns
print("Different columns in guide_df:", diff_col_in_guide)
print("Different columns in tourist_df:", diff_col_in_tourist)


Different columns in guide_df: {'Gender', 'Domicile', 'Name'}
Different columns in tourist_df: {'Genders', 'Names', 'Destinations'}


In [8]:
# Rename the diff column
guide_df = guide_df.rename(columns={'Name': 'Names'})
guide_df = guide_df.rename(columns={'Gender': 'Genders'})
guide_df = guide_df.rename(columns={'Domicile': 'Destinations'})


In [9]:
# Check if column names are the same
columns_equal = guide_df.columns.equals(tourist_df.columns)

if columns_equal:
    print("The DataFrames have the same column names.")
else:
    print("The DataFrames do not have the same column names.")

The DataFrames have the same column names.


In [10]:
user_df = pd.concat([tourist_df, guide_df])
user_df = user_df.reset_index(drop=True)
user_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,4,4,5,5,4,4,2,4,4,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,5,5,5,5,4,3,3,4,3,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,3,2,3,4,3,2,3,4,3,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,5,3,4,5,4,4,1,4,4,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,3,3,4,4,2,3,3,2,3,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Dera,1,0,1,0,0,0,0,0,0,...,4,5,5,5,4,5,5,5,5,guide
97,Salsabilla Rizki,1,0,1,0,0,0,0,0,1,...,5,5,5,5,5,5,2,3,4,guide
98,Salma,1,0,1,0,0,0,0,1,1,...,5,5,5,5,5,5,4,5,4,guide
99,Deva,1,0,0,1,0,0,0,0,1,...,4,4,3,4,3,4,2,4,3,guide


## Check Summary New Datasets

In [11]:
# Looking at the index of the columns
columns = user_df.columns
for idx, column in enumerate(columns):
    print(f"Column index: {idx}, Column name: {column}")

Column index: 0, Column name: Names
Column index: 1, Column name: Genders
Column index: 2, Column name: Destinations
Column index: 3, Column name: 17-25
Column index: 4, Column name: 26-34
Column index: 5, Column name: 35-43
Column index: 6, Column name: 44-52
Column index: 7, Column name: 52+
Column index: 8, Column name: Historical tours
Column index: 9, Column name: Adventure tours
Column index: 10, Column name: Nature and wildlife tours
Column index: 11, Column name: Culinary tours
Column index: 12, Column name: Wellness and retreat tours
Column index: 13, Column name: Architectural tours
Column index: 14, Column name: Educational tours
Column index: 15, Column name: Shopping tours
Column index: 16, Column name: EXT1
Column index: 17, Column name: EXT2
Column index: 18, Column name: EXT3
Column index: 19, Column name: EXT4
Column index: 20, Column name: EXT5
Column index: 21, Column name: EST1
Column index: 22, Column name: EST2
Column index: 23, Column name: EST3
Column index: 24,

In [12]:
user_df.describe()

,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
count,101.000000,101.000000,101.000000,101.000000,101.0,101.0,101.0,101.000000,101.000000,101.000000,...,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,0.702970,0.415842,0.702970,0.049505,0.0,0.0,0.0,0.386139,0.475248,0.653465,...,3.683168,3.396040,3.613861,3.811881,3.841584,3.297030,3.435644,2.910891,3.841584,3.445545
std,0.459229,0.495325,0.459229,0.218002,0.0,0.0,0.0,0.489291,0.501878,0.478239,...,1.009264,1.010734,1.009656,1.046068,0.945861,0.964827,0.973816,1.040183,0.977064,0.994739
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000
50%,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000
75%,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,4.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,3.000000,5.000000,4.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [13]:
# Check that we not missing any value after combine columns
print('Is there any missing value? ', user_df.isnull().values.any())
print('How many missing values? ', user_df.isnull().values.sum())

Is there any missing value?  False
How many missing values?  0


# **Concatenating Personality Type Prediction**

In [14]:
# Copy origin user data into match data to use in cluster
match_df = user_df.copy()

In [15]:
# Drop Column that not use for clustering match
match_df.drop(['Names', 'Destinations', 'Roles'], axis=1, inplace=True)
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,1,1,0,0,0,0,0,1,1,0,...,5,4,4,5,5,4,4,2,4,4
1,1,1,0,0,0,0,1,0,0,1,...,5,5,5,5,5,4,3,3,4,3
2,1,1,0,0,0,0,0,0,1,1,...,2,3,2,3,4,3,2,3,4,3
3,0,0,0,0,0,0,0,1,1,0,...,3,5,3,4,5,4,4,1,4,4
4,0,1,0,0,0,0,0,1,1,1,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,...,5,4,5,5,5,4,5,5,5,5
97,1,1,0,0,0,0,0,1,1,1,...,3,5,5,5,5,5,5,2,3,4
98,1,1,0,0,0,0,1,1,0,1,...,5,5,5,5,5,5,5,4,5,4
99,1,0,1,0,0,0,0,1,1,1,...,4,4,4,3,4,3,4,2,4,3


In [16]:
persona_test_result = match_df.iloc[:, -25:]
persona_test_result

,EXT1,EXT2,EXT3,EXT4,EXT5,EST1,EST2,EST3,EST4,EST5,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,4,2,4,4,3,3,3,1,2,3,...,5,4,4,5,5,4,4,2,4,4
1,3,3,1,1,2,2,4,4,5,3,...,5,5,5,5,5,4,3,3,4,3
2,3,4,2,1,2,3,2,2,4,3,...,2,3,2,3,4,3,2,3,4,3
3,3,4,5,3,2,1,1,1,1,1,...,3,5,3,4,5,4,4,1,4,4
4,4,4,4,5,2,3,4,4,3,2,...,4,3,3,4,4,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5,4,5,4,5,4,3,3,5,4,...,5,4,5,5,5,4,5,5,5,5
97,3,4,3,3,3,2,4,2,2,2,...,3,5,5,5,5,5,5,2,3,4
98,2,1,4,4,4,4,5,5,2,1,...,5,5,5,5,5,5,5,4,5,4
99,4,4,4,4,4,2,3,2,4,3,...,4,4,4,3,4,3,4,2,4,3


In [17]:
# Convert input data to float32
data_input = np.array(persona_test_result, dtype=np.float32)

# Load model
model_path = 'persona_model/1'
loaded_model = tf.saved_model.load(model_path)

# Get the default signature of the model
inference_fn = loaded_model.signatures["serving_default"]

# Use models to make persona type predictions
predictions = inference_fn(tf.constant(data_input))["dense_2"]

In [18]:
predicted_clusters = np.argmax(predictions, axis=1)
persona_type = pd.DataFrame({'Persona type': predicted_clusters})
persona_type 

,Persona type
0,2
1,1
2,3
3,2
4,4
...,...
96,2
97,2
98,0
99,0


In [19]:
# Personality type column is changed to one hot encoding
persona_type = pd.get_dummies(persona_type, columns=['Persona type'])
persona_type = persona_type.astype(int)
persona_type

,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,1,0,0
4,0,0,0,0,1
...,...,...,...,...,...
96,0,0,1,0,0
97,0,0,1,0,0
98,1,0,0,0,0
99,1,0,0,0,0


In [20]:
# Delete persona test result column in match_df
match_df = match_df.iloc[:, :-25]
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours
0,1,1,0,0,0,0,0,1,1,0,0,0,0,1
1,1,1,0,0,0,0,1,0,0,1,0,1,0,0
2,1,1,0,0,0,0,0,0,1,1,0,1,0,0
3,0,0,0,0,0,0,0,1,1,0,0,1,0,0
4,0,1,0,0,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,0,0,0,1
97,1,1,0,0,0,0,0,1,1,1,0,0,0,0
98,1,1,0,0,0,0,1,1,0,1,0,1,0,0
99,1,0,1,0,0,0,0,1,1,1,0,0,0,0


In [21]:
# Concatenate 'match_df' with 'persona_type'
match_df = pd.concat([match_df, persona_type], axis=1)
match_df

,Genders,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4
0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0
1,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
2,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0
97,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0
98,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0
99,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0


In [22]:
# Concatenate persona type to user_df
roles = user_df.iloc[:, -1:]
roles

,Roles
0,tourist
1,tourist
2,tourist
3,tourist
4,tourist
...,...
96,guide
97,guide
98,guide
99,guide


In [23]:
# Delete persona test result column in match_df
user_df = user_df.iloc[:, :-26]
user_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,Nature and wildlife tours,Culinary tours,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Dera,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1
97,Salsabilla Rizki,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0
98,Salma,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0
99,Deva,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0


In [24]:
# Concatenate 'match_df' with 'persona_type'
user_df = pd.concat([user_df, persona_type, roles], axis=1)
user_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Dera,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,guide
97,Salsabilla Rizki,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,guide
98,Salma,1,0,1,0,0,0,0,1,1,...,0,1,0,0,1,0,0,0,0,guide
99,Deva,1,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,guide


# **Make Matchmaking Model**

## Reduce Dimension from Data

In [25]:
match_array = match_df.values
match_array

array([[1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]], dtype=int64)

## Test Load Model

In [26]:
# Load the PCA model
loaded_model = tf.saved_model.load('pca_model/1/')

In [27]:
new_data_tf = tf.convert_to_tensor(match_array, dtype=tf.float32)

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Print the PCA value
print(pca_value)

tf.Tensor(
[[ 0.90362394  0.4586686 ]
 [-1.1234949  -0.5327303 ]
 [-0.20840554  0.09403273]
 [-0.00210786  1.41256   ]
 [ 0.96277475  0.27177203]
 [ 0.06641878  0.49033108]
 [-0.646333    0.5166402 ]
 [ 0.80000377 -0.7733567 ]
 [-1.1317303   0.01277947]
 [ 0.1640983  -0.32443506]
 [-0.03162198  0.72825146]
 [-0.21996841  1.42571   ]
 [-0.8959171  -0.5676879 ]
 [-0.68994904 -0.5651168 ]
 [-0.56146425  0.03711636]
 [-1.0953424   0.23444465]
 [ 0.33884782  0.09858982]
 [ 0.00608534 -0.35555592]
 [-0.51399726  0.15410763]
 [-0.6629581   0.41797698]
 [-0.7578075  -0.7598223 ]
 [-0.49729478 -0.43017465]
 [-1.0476477   0.31849688]
 [ 0.9739295  -0.16895357]
 [ 0.5684941  -0.5919983 ]
 [-1.3624796  -0.00943238]
 [ 0.08562335  1.3656352 ]
 [-0.5247066   0.07196948]
 [ 0.09501052 -0.08412665]
 [-0.32602304 -0.86611736]
 [-0.53399134  0.19448036]
 [ 0.8953172   0.06471789]
 [-0.00255138  0.15033323]
 [ 0.6424954  -0.7136053 ]
 [ 0.46385843 -0.40493926]
 [-1.4939647   0.09693141]
 [-0.17456031  0.

In [28]:
pca_value_arr = pca_value.numpy()
pca_value_arr

array([[ 0.90362394,  0.4586686 ],
       [-1.1234949 , -0.5327303 ],
       [-0.20840554,  0.09403273],
       [-0.00210786,  1.41256   ],
       [ 0.96277475,  0.27177203],
       [ 0.06641878,  0.49033108],
       [-0.646333  ,  0.5166402 ],
       [ 0.80000377, -0.7733567 ],
       [-1.1317303 ,  0.01277947],
       [ 0.1640983 , -0.32443506],
       [-0.03162198,  0.72825146],
       [-0.21996841,  1.42571   ],
       [-0.8959171 , -0.5676879 ],
       [-0.68994904, -0.5651168 ],
       [-0.56146425,  0.03711636],
       [-1.0953424 ,  0.23444465],
       [ 0.33884782,  0.09858982],
       [ 0.00608534, -0.35555592],
       [-0.51399726,  0.15410763],
       [-0.6629581 ,  0.41797698],
       [-0.7578075 , -0.7598223 ],
       [-0.49729478, -0.43017465],
       [-1.0476477 ,  0.31849688],
       [ 0.9739295 , -0.16895357],
       [ 0.5684941 , -0.5919983 ],
       [-1.3624796 , -0.00943238],
       [ 0.08562335,  1.3656352 ],
       [-0.5247066 ,  0.07196948],
       [ 0.09501052,

In [29]:
# Looking at the index of the columns
columns = user_df.columns
for idx, column in enumerate(columns):
    print(f"Column index: {idx}, Column name: {column}")

Column index: 0, Column name: Names
Column index: 1, Column name: Genders
Column index: 2, Column name: Destinations
Column index: 3, Column name: 17-25
Column index: 4, Column name: 26-34
Column index: 5, Column name: 35-43
Column index: 6, Column name: 44-52
Column index: 7, Column name: 52+
Column index: 8, Column name: Historical tours
Column index: 9, Column name: Adventure tours
Column index: 10, Column name: Nature and wildlife tours
Column index: 11, Column name: Culinary tours
Column index: 12, Column name: Wellness and retreat tours
Column index: 13, Column name: Architectural tours
Column index: 14, Column name: Educational tours
Column index: 15, Column name: Shopping tours
Column index: 16, Column name: Persona type_0
Column index: 17, Column name: Persona type_1
Column index: 18, Column name: Persona type_2
Column index: 19, Column name: Persona type_3
Column index: 20, Column name: Persona type_4
Column index: 21, Column name: Roles


In [30]:
def matchmaking(user_id):
   # Get user destination
    user_destination = user_df.loc[user_id, 'Destinations']

    # Filter users with the same destination and roles value of 'guide'
    filtered_users = user_df[(user_df['Destinations'] == user_destination) & (user_df['Roles'] == 'guide')]

    # Get the index values of the filtered users
    filtered_user_indices = filtered_users.index

    distances = []
    # print("user", [pca_value_arr[user_id]])
    for idx in filtered_user_indices:
        # Get the distances within the cluster of the specified user
        distance_idx = euclidean_distances([pca_value_arr[user_id]], [pca_value_arr[idx]])
        # print(idx, [pca_value_arr[idx]])
        distances.append(distance_idx)

    # Normalize the distances within the range of 0-100
    normalized_distances = 1 - distances / np.max(distances)
    scores = normalized_distances * 100
    
    # Reshape the list
    scores = np.reshape(scores, (len(filtered_user_indices),))
    
    matches = [(index, score) for index, score in enumerate(scores) if index != user_id]
    
    # Sort the matches based on the highest score
    matches.sort(key=lambda x: x[1], reverse=True)

    print("User Index:", user_id)
    print("Matched Peers:")
    match_idx = [user_id]
    for match_index, score in matches:
        if user_df.iloc[filtered_user_indices[match_index], 21] == 'guide':
            if user_df.iloc[filtered_user_indices[match_index], 2] == user_df.iloc[user_id, 2]:
                match_idx.append(filtered_user_indices[match_index])
                print("Index:", filtered_user_indices[match_index], "| Score:", f"{score:.2f}%")
                
    # Check if Destination and roles is correct 
    print()         
    print(user_df.iloc[match_idx, [0, 1, 2, 21]])
    
    # Check if they get same preference
    print()
    print(user_df.iloc[match_idx, 8:16])
    
    # Check the personality
    print()
    print(user_df.iloc[match_idx, 16:21])

In [31]:
matchmaking(0)

User Index: 0
Matched Peers:
Index: 97 | Score: 89.73%
Index: 83 | Score: 84.38%
Index: 94 | Score: 83.81%
Index: 76 | Score: 81.29%
Index: 74 | Score: 70.32%
Index: 99 | Score: 68.93%
Index: 100 | Score: 62.79%
Index: 95 | Score: 61.61%
Index: 96 | Score: 56.81%
Index: 92 | Score: 45.98%
Index: 85 | Score: 45.30%
Index: 75 | Score: 43.50%
Index: 88 | Score: 41.18%
Index: 89 | Score: 40.32%
Index: 93 | Score: 40.24%
Index: 90 | Score: 37.18%
Index: 91 | Score: 33.41%
Index: 98 | Score: 28.94%
Index: 87 | Score: 26.20%
Index: 82 | Score: 0.00%

                         Names  Genders  Destinations    Roles
0       Novia Rizki Wulandari         1             0  tourist
97            Salsabilla Rizki        1             0    guide
83                      Elma N        1             0    guide
94                           E        1             0    guide
76               Abiyyu Farhan        0             0    guide
74                  Salsabilla        1             0    guide
99       

In [32]:
matchmaking(8)

User Index: 8
Matched Peers:
Index: 80 | Score: 46.64%
Index: 84 | Score: 36.87%
Index: 86 | Score: 33.73%
Index: 79 | Score: 21.82%
Index: 81 | Score: 18.76%
Index: 78 | Score: 9.41%
Index: 77 | Score: 0.00%

                  Names  Genders  Destinations    Roles
8     Dimas Alfiansyah         0             1  tourist
80        Muthiah Hanun        1             1    guide
84                 Hani        1             1    guide
86              fadilah        1             1    guide
79               Andani        1             1    guide
81  Ihlasul Mufti Faqih        0             1    guide
78               Faizah        1             1    guide
77     Rahmaliyah Kadir        1             1    guide

    Historical tours  Adventure tours  Nature and wildlife tours   
8                  1                0                          0  \
80                 0                0                          0   
84                 0                0                          1   
86           

# **Test on New User**

In [33]:
new_user = ['Joko', 0, 0, 0,1,0,0,0, 0,1,0,1,0,0,1,0, 5,5,4,4,4,5,4,4,5,5,4,4,2,4,4,3,4,2,1,2,3,2,2,4,3, 'tourist']

In [34]:
len(new_user)

42

## Predict New User Personality Type

In [35]:
# Pick the result of personality test
new_user_persona_test = new_user[16:41]

# Convert to dataframe 
new_user_persona_test = pd.DataFrame([new_user_persona_test ])
new_user_persona_test

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,5,5,4,4,4,5,4,4,5,5,...,3,4,2,1,2,3,2,2,4,3


In [36]:
# Convert input data to float32
data_input = np.array(new_user_persona_test, dtype=np.float32)

# Load model
model_path = 'persona_model/1'
loaded_model = tf.saved_model.load(model_path)

# Get the default signature of the model
inference_fn = loaded_model.signatures["serving_default"]

# Use models to make persona type predictions
predictions = inference_fn(tf.constant(data_input))["dense_2"]

In [37]:
new_user_persona_type = np.argmax(predictions, axis=1)
new_user_persona_type = new_user_persona_type.tolist()

binary_list = [[int(i == val) for i in range(5)] for val in new_user_persona_type]
output_list = binary_list[0]
output_list 

[0, 1, 0, 0, 0]

## Append 

In [38]:
# Delete hasil personality survey pada new_user list
del new_user[16:41]

In [39]:
# Menyisipkan output_list sebelum elemen 'tourist'
new_user = new_user[:-1] + output_list + new_user[-1:]
new_user

['Joko', 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 'tourist']

In [40]:
# Add the new row using loc indexer
user_df.loc[len(user_df)] = new_user
user_df

,Names,Genders,Destinations,17-25,26-34,35-43,44-52,52+,Historical tours,Adventure tours,...,Wellness and retreat tours,Architectural tours,Educational tours,Shopping tours,Persona type_0,Persona type_1,Persona type_2,Persona type_3,Persona type_4,Roles
0,Novia Rizki Wulandari,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,tourist
1,Herlina Kusyanuri Putri,1,0,1,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,tourist
2,Rifdah Alyaa,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,tourist
3,Alexis Purnomo,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,tourist
4,Rifky Surya Pratama,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,tourist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Salsabilla Rizki,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,guide
98,Salma,1,0,1,0,0,0,0,1,1,...,0,1,0,0,1,0,0,0,0,guide
99,Deva,1,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,guide
100,Dzaky,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,1,0,0,guide


In [41]:
# convert list new_user menjadi array dan memilih value yg akan digunakan untuk matchmaking
new_user_match = new_user[1:2] + new_user[3:21]
new_user_match

[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]

In [42]:
new_data_tf = tf.convert_to_tensor(new_user_match, dtype=tf.float32)
new_data_tf = tf.expand_dims(new_data_tf, axis=0)  # Adding dimention batch

loaded_model = tf.saved_model.load('pca_model/1/')

# Predict the PCA value using the loaded model
pca_value = loaded_model.apply_pca(new_data_tf)

# Convert pca_value into array
new_user_pca_value = pca_value.numpy()

# Print the PCA value
print(new_user_pca_value)

[[-1.6472878e+00  2.3469329e-07]]


In [43]:
# Append pca value of new user to an array contains pca value from stored data 
pca_value_arr = np.append(pca_value_arr, new_user_pca_value, axis=0)
pca_value_arr

array([[ 9.0362394e-01,  4.5866859e-01],
       [-1.1234949e+00, -5.3273028e-01],
       [-2.0840554e-01,  9.4032727e-02],
       [-2.1078587e-03,  1.4125600e+00],
       [ 9.6277475e-01,  2.7177203e-01],
       [ 6.6418782e-02,  4.9033108e-01],
       [-6.4633298e-01,  5.1664019e-01],
       [ 8.0000377e-01, -7.7335668e-01],
       [-1.1317303e+00,  1.2779474e-02],
       [ 1.6409829e-01, -3.2443506e-01],
       [-3.1621978e-02,  7.2825146e-01],
       [-2.1996841e-01,  1.4257100e+00],
       [-8.9591712e-01, -5.6768793e-01],
       [-6.8994904e-01, -5.6511682e-01],
       [-5.6146425e-01,  3.7116364e-02],
       [-1.0953424e+00,  2.3444465e-01],
       [ 3.3884782e-01,  9.8589823e-02],
       [ 6.0853362e-03, -3.5555592e-01],
       [-5.1399726e-01,  1.5410763e-01],
       [-6.6295809e-01,  4.1797698e-01],
       [-7.5780749e-01, -7.5982231e-01],
       [-4.9729478e-01, -4.3017465e-01],
       [-1.0476477e+00,  3.1849688e-01],
       [ 9.7392952e-01, -1.6895357e-01],
       [ 5.68494

# Matchmaking New User

In [44]:
matchmaking(101)

User Index: 101
Matched Peers:
Index: 82 | Score: 69.65%
Index: 98 | Score: 59.71%
Index: 87 | Score: 47.70%
Index: 91 | Score: 44.04%
Index: 75 | Score: 42.91%
Index: 85 | Score: 42.45%
Index: 92 | Score: 37.32%
Index: 73 | Score: 27.89%
Index: 95 | Score: 27.48%
Index: 93 | Score: 24.05%
Index: 99 | Score: 22.18%
Index: 100 | Score: 20.45%
Index: 74 | Score: 19.23%
Index: 88 | Score: 17.19%
Index: 96 | Score: 10.71%
Index: 83 | Score: 8.31%
Index: 89 | Score: 6.56%
Index: 94 | Score: 5.93%
Index: 76 | Score: 3.95%
Index: 97 | Score: 2.48%
Index: 90 | Score: 0.00%

                         Names  Genders  Destinations    Roles
101                       Joko        0             0  tourist
82   Dessylva Maharany Santosa        1             0    guide
98                       Salma        1             0    guide
87                       Winda        1             0    guide
91                zahra kamila        1             0    guide
75                     Maulani        1          